# Machine Learning - CatBoost

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [2]:
required_libs = [ ("numpy", "numpy"),
                 ("pandas", "pandas"),
                 ("seaborn", "seaborn"),
                 ("matplotlib", "matplotlib"),
                 ("catboost", "catboost"),
                 ("sklearn", "sklearn"),
                 ("ipywidgets", "ipywidgets"),
                 ("shap", "shap"),
                ]

In [3]:
def is_lib_exists(name):
    import importlib
    lib = importlib.util.find_spec(name)
    return lib is not None

In [4]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
Installing catboost
     |████████████████████████████████| 64.8MB 61kB/s 
sklearn exists
ipywidgets exists
Installing shap
     |████████████████████████████████| 276kB 2.8MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394114 sha256=39a84a87c4f32235ca9c100ff2dd3704afbea8b241c383502bdb3ec840b7147c
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [5]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_d2_v3'

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import traceback
import catboost
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
from catboost import *
from catboost import datasets
np.set_printoptions(precision=4)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
%matplotlib inline
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [8]:
from IPython.display import display

In [9]:
# check catboost version
print(catboost.__version__)
!python --version

0.23.2
Python 3.6.9


In [10]:
# colab setup
if COLAB:
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects
  
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .
  !cp dl-projects/plot* .
  
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
  !ls -l gdrive/"My Drive"/datasets/ --block-size=M

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.ft"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

Cloning into 'dl-projects'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 2126 (delta 91), reused 32 (delta 14), pack-reused 1977
Receiving objects: 100% (2126/2126), 79.28 MiB | 12.27 MiB/s, done.
Resolving deltas: 100% (1316/1316), done.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
total 4948M
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr.ft
-rw------- 1 root root   17M Jul 

## EDA

In [11]:
from preprocess import *

In [12]:
data = pd.read_feather(DATASET)
origin_data = format_tabular(ORIGIN_DATASET)

In [13]:
jan_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jan.ft")
feb_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_feb.ft")
mar_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_mar.ft")
apr_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_apr.ft")
may_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_may.ft")
jun_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jun.ft")
jul_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jul.ft")

In [14]:
# Compare July data
jul_2020_orig = pd.read_pickle(DATASET_PATH/f"feature_matrix_2020_jul_orig.pkl")

In [15]:
# Compare July data
jul_2020.shape, jul_2020_orig.shape
display(jul_2020.columns.difference(jul_2020_orig.columns))
display(jul_2020_orig.columns.difference(jul_2020.columns))

((10000, 217), (10000, 213))

Index(['LAST(Results.PrizeType)_Prize', 'MODE(Results.PrizeType)_Prize',
       'NumberId', 'month', 'time', 'year'],
      dtype='object')

Index(['LAST(Results.PrizeType)', 'MODE(Results.PrizeType)'], dtype='object')

In [19]:
#jul_2020_orig.dtypes.head(10)
jul_2020_orig.shape
display(jul_2020_orig["LAST(Results.PrizeType)"].head(300))
len(list(jul_2020_orig["LAST(Results.PrizeType)"].values))

(10000, 213)

NumberId  time      
115       2020-07-01    Prize
204       2020-07-01    Prize
318       2020-07-01    Prize
513       2020-07-01    Prize
793       2020-07-01    Prize
                        ...  
9285      2020-07-01    Prize
8726      2020-07-01    Prize
93        2020-07-01    Prize
6128      2020-07-01    Prize
1096      2020-07-01    Prize
Name: LAST(Results.PrizeType), Length: 300, dtype: object

10000